## Regression exercises

Linear regression is one of the most important modelling techniques. We looked into some of the important concepts like fitting a linear model, how variation os explained by the models, error of the estimates and little bit of multiple and multivariate regression. 

In this exercise we will work with online news popularity dataset. The dataset has 61 attributes: 58 predictive attributes, 2 non-predictive, 1 response variable. We have to predict the number of shares for the news article using rest of the predictor variables. Lets start by reading the data into a datframe called cpu_data.

In [1]:
cpu_data = read.csv("../../../datasets/cpu performance/performance_data.txt")
head(cpu_data)

,adviser,X32.60,X125,X256,X6000,X256.1,X16,X128,X198,X199
1,amdahl,470v/7,29,8000,32000,32,8,32,269,253
2,amdahl,470v/7a,29,8000,32000,32,8,32,220,253
3,amdahl,470v/7b,29,8000,32000,32,8,32,172,253
4,amdahl,470v/7c,29,8000,16000,32,8,16,132,132
5,amdahl,470v/b,26,8000,32000,64,8,32,318,290
6,amdahl,580-5840,23,16000,32000,64,16,32,367,381


The input dataset does not have headers. 

**Activity 1: ** Grab the headers information from readme file and assign them to column sof cpu_data 

In [3]:
names=c("vendor_name","Model_Name","MYCT","MMIN","MMAX","CACH","CHMIN","CHMAX","PRP","ERP")
names(cpu_data)=names
head(cpu_data)

,vendor_name,Model_Name,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP,ERP
1,amdahl,470v/7,29,8000,32000,32,8,32,269,253
2,amdahl,470v/7a,29,8000,32000,32,8,32,220,253
3,amdahl,470v/7b,29,8000,32000,32,8,32,172,253
4,amdahl,470v/7c,29,8000,16000,32,8,16,132,132
5,amdahl,470v/b,26,8000,32000,64,8,32,318,290
6,amdahl,580-5840,23,16000,32000,64,16,32,367,381


The variable of interest here is `ERP`(estimated relative performance). Lets how the data is distributed in the dataset.  

In [4]:
summary(cpu_data)

    vendor_name       Model_Name       MYCT             MMIN      
 ibm      : 32   100       :  1   Min.   :  17.0   Min.   :   64  
 nas      : 19   1100/61-h1:  1   1st Qu.:  50.0   1st Qu.:  768  
 honeywell: 13   1100/81   :  1   Median : 110.0   Median : 2000  
 ncr      : 13   1100/82   :  1   Mean   : 204.2   Mean   : 2881  
 sperry   : 13   1100/83   :  1   3rd Qu.: 225.0   3rd Qu.: 4000  
 siemens  : 12   1100/84   :  1   Max.   :1500.0   Max.   :32000  
 (Other)  :106   (Other)   :202                                   
      MMAX            CACH           CHMIN            CHMAX       
 Min.   :   64   Min.   :  0.0   Min.   : 0.000   Min.   :  0.00  
 1st Qu.: 4000   1st Qu.:  0.0   1st Qu.: 1.000   1st Qu.:  5.00  
 Median : 8000   Median :  8.0   Median : 2.000   Median :  8.00  
 Mean   :11824   Mean   : 24.1   Mean   : 4.644   Mean   : 17.74  
 3rd Qu.:16000   3rd Qu.: 32.0   3rd Qu.: 6.000   3rd Qu.: 24.00  
 Max.   :64000   Max.   :256.0   Max.   :52.000   Max.   :176.

Some of the variables seems to have skewed distribution. Lets check using skewness function in 'e1071' package.

In [73]:
library(e1071)
skewness(cpu_data$MYCT)
skewness(cpu_data$MMIN)
skewness(cpu_data$MMAX)
skewness(cpu_data$CACH)
skewness(cpu_data$CHMIN)
skewness(cpu_data$CHMAX)
skewness(cpu_data$PRP)

[1] 2.499116

[1] 3.460406

[1] 2.102134

[1] 2.541911

[1] 4.041982

[1] 3.699302

[1] 3.844285

All the variables have large positive skews. We will plot the same and visually inspect the skewed nature of data. 

In [75]:
# temp=subset(cpu_data,select=c(3:10))
# temp$MYCT=as.numeric(temp$MYCT)
# temp$MMIN=as.numeric(temp$MMIN)
# temp$MMAX=as.numeric(temp$MMAX)
# temp$CACH=as.numeric(temp$CACH)
# temp$CHMIN=as.numeric(temp$CHMIN)
# temp$CHMAX=as.numeric(temp$CHMAX)
# temp$PRP=as.numeric(temp$PRP)
# temp$ERP=as.numeric(temp$ERP)

# sapply(cpu_data,class)
# head(temp)
cor(cpu_data)

ERROR: Error in cor(cpu_data): 'x' must be numeric


In [77]:
# library(gridExtra)
# library(ggplot2)
# grid.arrange(
# qplot(cpu_data$MYCT,geom="histogram",bins=25),
# qplot(cpu_data$MMIN,geom="histogram",bins=30),
# qplot(cpu_data$MMAX,geom="histogram",bins=25),
# # qplot(cpu_data$CACH,geom="histogram",bins=25),
#     qplot(cpu_data$CHMIN,geom="histogram",bins=25),
#     qplot(cpu_data$CHMAX,geom="histogram",bins=25),
#     qplot(cpu_data$PRP,geom="histogram",bins=25),
# nrow=3,ncol=3)

We can see all the variables have a positively skewed distribution. We can transform the data to handle the skewness. It’s generally a good idea to log transform data when values range over several orders of magnitude. Also, modeling techniques often have a difficult time with very wide data ranges. Taking the logarithm only works if the data is non-negative. If you go through the summary of cpu_data all the variables have positive values. 

So lets apply log transformations to data to normalize it and plot the histograms.

In [33]:
x=cpu_data$CACH
log_CACH <- function(x) { asinh(x/2)/log(10) }
nrow(log_CACH)
log_MYCT=log(cpu_data$MYCT)
log_MMIN=log(cpu_data$MMIN)
log_MMAX=log(cpu_data$MMAX)
# log_CACH=log(cpu_data$CACH)
# log_CHMIN=log(cpu_data$CHMIN)
# log_CHMAX=log(cpu_data$CHMAX)
log_PRP=log(cpu_data$PRP)
log_data=data.frame("vendor_name"=cpu_data$vendor_name,"Model_Name"=cpu_data$Model_Name,"MYCT"=log_MYCT,"MMIN"=log_MMIN,
                    "MMAX"=log_MMAX,"CACH"=log_CACH,"PRP"=log_PRP,"ERP"=cpu_data$ERP)
# ,"CHMIN"=log_CHMIN,"CHMAX"=log_CHMAX
grid.arrange(
    qplot(log_MYCT,geom="histogram",bins=25),
    qplot(log_MMIN,geom="histogram",bins=25),
    qplot(log_MMAX,geom="histogram",bins=25),
    qplot(log_CACH,geom="histogram",bins=25),
    qplot(log_CHMIN,geom="histogram",bins=25),
    qplot(log_CHMAX,geom="histogram",bins=25),
    qplot(log_PRP,geom="histogram",bins=25),
    nrow=3,ncol=3)

# head(log_MYCT,50)

NULL

ERROR: Error in data.frame(vendor_name = cpu_data$vendor_name, Model_Name = cpu_data$Model_Name, : arguments imply differing number of rows: 208, 0


In [25]:
summary(log_data)

    vendor_name       Model_Name       MYCT            MMIN       
 ibm      : 32   100       :  1   Min.   :2.833   Min.   : 4.159  
 nas      : 19   1100/61-h1:  1   1st Qu.:3.912   1st Qu.: 6.644  
 honeywell: 13   1100/81   :  1   Median :4.700   Median : 7.601  
 ncr      : 13   1100/82   :  1   Mean   :4.747   Mean   : 7.369  
 sperry   : 13   1100/83   :  1   3rd Qu.:5.416   3rd Qu.: 8.294  
 siemens  : 12   1100/84   :  1   Max.   :7.313   Max.   :10.373  
 (Other)  :106   (Other)   :202                                   
      MMAX             CACH           CHMIN            CHMAX      
 Min.   : 4.159   Min.   : -Inf   Min.   :  -Inf   Min.   : -Inf  
 1st Qu.: 8.294   1st Qu.: -Inf   1st Qu.:0.0000   1st Qu.:1.609  
 Median : 8.987   Median :2.079   Median :0.6931   Median :2.079  
 Mean   : 8.923   Mean   : -Inf   Mean   :  -Inf   Mean   : -Inf  
 3rd Qu.: 9.680   3rd Qu.:3.466   3rd Qu.:1.7918   3rd Qu.:3.178  
 Max.   :11.067   Max.   :5.545   Max.   :3.9512   Max.   :5.1

In [45]:
cpu_reg=lm(ERP ~ . -vendor_name -Model_Name, data=cpu_data)
summary(cpu_reg)


Call:
lm(formula = ERP ~ . - vendor_name - Model_Name, data = cpu_data)

Residuals:
     Min       1Q   Median       3Q      Max 
-115.703   -9.687    2.698   15.075  180.069 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -3.392e+01  4.763e+00  -7.121 1.89e-11 ***
MYCT         3.728e-02  9.478e-03   3.933 0.000116 ***
MMIN         5.450e-03  1.123e-03   4.853 2.45e-06 ***
MMAX         3.400e-03  3.997e-04   8.505 4.22e-15 ***
CACH         9.648e-02  8.862e-02   1.089 0.277597    
CHMIN        1.894e-02  4.561e-01   0.042 0.966919    
CHMAX        3.180e-01  1.357e-01   2.343 0.020124 *  
PRP          5.819e-01  3.798e-02  15.321  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 31.75 on 200 degrees of freedom
Multiple R-squared:  0.9595,	Adjusted R-squared:  0.958 
F-statistic: 676.2 on 7 and 200 DF,  p-value: < 2.2e-16


In [29]:
log(0.1)

[1] -2.302585